# Pumping test interpretation: Cooper-Jacob Method

## Importing the libraries

In [1]:
#-- Check and install required packages if not already installed 
import sys
import subprocess

def if_require(package):
    try:
        __import__(package)
    except ImportError:
        print(f"{package} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        
#-- Install packages only if they aren't already installed 
if_require("ipywidgets")
if_require("openpyxl")
if_require("xlrd")

#-- Import the rest of libraries
import matplotlib.pyplot as plt
import ipywidgets as widgets
import seaborn as sns
import pandas as pd
import numpy as np
import scipy as sp
import traitlets
import operator
import os

from IPython.display import display, clear_output, HTML
from scipy.optimize import fmin_l_bfgs_b
from ipywidgets import Layout, Output
from tkinter import Tk, filedialog
from functools import reduce
from numpy.linalg import inv

import warnings
warnings.filterwarnings("ignore")

## Widgets and method definition

In [2]:
#------------------------------------------#
# WIDGETS DEFINITION                       #
#------------------------------------------#
def create_widget(description: str, style=None, width=None):
    """
    Function for the generation of widget 

    Parameters:
        - description (str): description of the widget
    Return:
        - widget_value: returns the value of the widget
    """ 
    widget_value = widgets.Text(value='', description=description, 
                                style={'description_width': 'initial'}, width = '500px')
    return widget_value


def update_numeric_value(widget, variable):
    """Observes changes in the widget's value and updates the variable accordingly.

    When a user enters or modifies text in the widget, the 'update_value' function 
    will be automatically called, allowing you to respond to and handle the changes 
    in the widget's value. 

    Parameters
    ----------
    widget : ipywidgets.Widget
        The widget object that will be observed for changes in its value
    variable : Any
        A global variable to be updated based on the widget's input

    Returns
    -------
    widget : ipywidgets.Widget
        The original widget, with an observer attached to monitor changes

    Notes
    -----
    If the new value in the widget is blank, 'variable' will be set to None. If the 
    input is invalid, an error message will be displayed
    """
    def update_value(change):
        global variable
        new_value = change.new.strip()
        if new_value == '':
            variable = None
        else:
            try:
                variable = float(new_value)
            except ValueError:
                print(f"Invalid input for '{widget.description}', please enter a valid value.")

    widget.observe(update_value, names='value')
    return widget

def Q_input(Q):
    """Widget definition for flow rate.
    """
    Q_wid = create_widget(description='Flow rate [L³/T]:',
                           style={'description_width': 'initial'}, width='500px')  
    Q_wid = update_numeric_value(Q_wid, Q)
    return Q_wid


def r_input(r):
    """Widget definition for the distance from observation point to pumping well. 
    """
    r_wid = create_widget(description='Distance from obs. point to well [L]:',
                           style={'description_width': 'initial'}, width='500px')  
    r_wid = update_numeric_value(r_wid, r)
    return r_wid


#------------------------------------------#
# METHODS DEFINITION                       #
#------------------------------------------#
#--Cooper-Jacob
def dotproduct(vec1:list, vec2:list) -> float:
    return sum(map(operator.mul, vec1, vec2))

def parameters_line(n, sum_tjac, sumsq_tjac, sum_sjac, sumprod_ts):
        
    matrix1 = [[n, sum_tjac], 
              [sum_tjac, sumsq_tjac]]
    matrix2 = [sum_sjac, sumprod_ts]
          
    intercept, slope = np.matmul(inv(matrix1), matrix2)
    
    return intercept, slope

def cooper_jacob_method(Q:float, r:float, time:list, t_mod:list) -> list:
       
    #--Time information
    n = len(t_mod)
    in_time = t_mod[0]
    end_time = t_mod[-1]
    idx_in_time = np.where(time == in_time)[0][0]
    idx_end_time = np.where(time == end_time)[0][0]
       
    #--Jacob time and drawdown
    logt_jac = [np.log10(t_mod[i]) for i in range(n)]
    s_jac = drawdown[idx_in_time:idx_end_time+1]
    
    sum_tjac = np.sum(logt_jac)    
    sumsq_tjac = sum(i*i for i in logt_jac)
    sum_sjac = np.sum(s_jac)
    
    sumprod_ts = dotproduct(logt_jac, s_jac)
    
    intercept, slope = parameters_line(n, sum_tjac, sumsq_tjac, sum_sjac, sumprod_ts)

    #--Jacob fitting
    x1 = 10**(-intercept/slope)
    y1 = intercept + slope * np.log10(x1)
    x2 = 1.1 * time[-1]
    y2 = intercept + slope * np.log10(x2)
    
    #--Transmissivity and Storage
    m = np.log(10)/(4*np.pi)  
    T = m*(Q/slope)
    S = (2.25*T*x1)/(r**2)      
        
    return intercept, slope, T, S, x1, y1, x2, y2

## Importing the data
**NOTE**: Find and load the file ``leakyAq_example_data.txt``. For future use of this code with real or additional scenarios, please ensure that your input files follow the same structure as the provided text file:

-   First row: brief description of the dataset.
-   Second row: column headers for the parameters.
-   Following rows: the numerical data.

In [ ]:
#--Initialize a DataFrame and variables to use afterward
data = pd.DataFrame() 

class SelectFileButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog.
    """
    
    def __init__(self):
        super(SelectFileButton, self).__init__()      
        #--Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        #--Create the button.
        self.description = "Select Files"
        self.icon = "square-o"
        self.style.button_color = "orange"
        #--Set on click behavior.
        self.on_click(self.select_files)
          
    def select_files(self, b):
        """Generate instance of tkinter.filedialog.
    
        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        
        global time, drawdown, weights
        
        with out:
            try:
                #--Create Tk root
                root = Tk()
                #--Hide the main window
                root.withdraw()
                #--Raise the root to the top of all windows.
                root.call('wm', 'attributes', '.', '-topmost', True)
                #--List of selected files will be set to b.value
                selected_files = filedialog.askopenfilename(multiple=True)
                if selected_files:
                    b.files = selected_files
                    b.description = "Files Selected"
                    b.icon = "check-square-o"
                    b.style.button_color = "lightgreen"
                    file_name, file_extension = os.path.splitext(b.files[0])
                    if file_extension != '.txt' and file_extension != '.csv':
                        print("Your selected file is neither a CSV nor an XLSX file. Please select a correct input file.")
                    else:
                        print(f"You selected {os.path.basename(b.files[0])} file as your input observation data.")        
                        raw_data = pd.read_csv(b.files[0], sep="\t", skiprows=1)
                        data['Time [d]'] = pd.to_numeric(raw_data.iloc[1:, 0].reset_index(drop=True))
                        data['Drawdown [m]'] = pd.to_numeric(raw_data.iloc[1:, 1].reset_index(drop=True))
                        data['Weights'] = np.ones(len(data['Time [d]']))   
                        time = data['Time [d]'].to_numpy()
                        drawdown = data['Drawdown [m]'].to_numpy()
                        weights = data['Weights']
                        
                        #------------------------------------------#
                        # Plot the raw data                        #
                        #------------------------------------------#                      
                        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

                        # ax1: arihtmetic vs arithmetic   
                        ax1.scatter(time, drawdown, linewidth=1)
                        ax1.set_xlabel('time [d]')
                        ax1.set_ylabel('Drawdown [m]')
                        
                        # ax2: arithmetic vs log (semilog)
                        ax2.scatter(time, drawdown, linewidth=1)
                        ax2.set_xlabel('time [d]')
                        ax2.set_ylabel('Drawdown [m]')
                        ax2.set_xscale('log')

                        # ax2: log-log
                        ax3.scatter(time, drawdown, linewidth=1) 
                        ax3.set_xlabel('time [d]')
                        ax3.set_ylabel('Drawdown [m]')
                        ax3.set_yscale('log')
                        ax3.set_xscale('log')

                        plt.tight_layout()
                        plt.show()                          

                else:
                    print("No file selected.")
            except:
                pass
        
        def get_input_file(self): 
            """Returns the selected file."""
            return self.observations

out = widgets.Output()
input_file = SelectFileButton()
widgets.VBox([input_file, out])

## Cooper-Jacob method

🔹 Known Parameters
- **Flow rate (Q):** 1728 m²/d  
- **Distance from observation point to well (r):** 0.1 m  

In [ ]:
#-- Button to trigger the calculation
output = widgets.Output()
button = widgets.Button(button_style='info', 
                        description="Click to solve Cooper-Jacob method",
                        tooltip='Click me', icon='check', 
                        layout=widgets.Layout(width='300px', height='30px'))

def on_button_clicked(button):
    Q = float(Q_wid.value) 
    r = float(r_wid.value) 

    #--Find the indices for selected time range
    in_time = np.where(time == sel_initial_time.value)[0][0]
    end_time = np.where(time == sel_ending_time.value)[0][0]+1
    time_mod = time[in_time:end_time]
    
    intercept, slope, T, S, x1, y1, x2, y2 = cooper_jacob_method(Q, r, time, time_mod)
           
    with output:
        output.clear_output(wait=True)        
        
        print(f"Storage coefficient (S): {S:.4f}")
        print(f"Transmissivity (T): {T:.2f}")
               
        #-- Plot the figure
        fig, (ax1) = plt.subplots(1, 1, figsize=(5, 5))

        # ax1: arithmetic vs log (semilog)
        ax1.scatter(time, drawdown, linewidth=1, edgecolors='blue', facecolors = "none")  
        ax1.scatter(time[in_time:end_time], drawdown[in_time:end_time], 
                    linewidth=1, edgecolors='blue', facecolors="limegreen",
                    label="Times used for calibration")       
        ax1.set_xlabel('time [d]')
        ax1.set_ylabel('Drawdown [m]')
        ax1.plot([x1, x2], [y1, y2], color="red", label="Cooper—Jacob method")
        ax1.set_xscale('log')
        ax1.legend(loc='lower right')
        ax1.set_xlim(time[0]-0.0003, time[-1]+1)

        plt.tight_layout()
        plt.show()
 
#-- Attaching the function to the button's click event
button.on_click(on_button_clicked)

#-- Displaying the widgets and button
Q_wid = Q_input(Q=0)  
r_wid = r_input(r=0)

initial_time = time
ending_time = time           
sel_initial_time = widgets.Dropdown(options=time, value=time[0], 
                          description="Initial time")
sel_ending_time = widgets.Dropdown(options=time, value=time[-1], 
                           description="Ending time")

#-- Defining the order of widgets
widgets_list = [Q_wid, r_wid, sel_initial_time, sel_ending_time, button]

#-- Displaying the widgets
display(widgets.VBox(widgets_list), output)

Output()